#### Assign parent / child relationships
- P: Parent
- C: Child
- Co: Co-completed
- S: Standalone
- Example: P|C = parent to the well on the left side and child to the well on the right side
- Example: |Co = Standalone on the left side and co-completed with the well to the right

In [ ]:
import pandas as pd
import numpy as np
from config.config_loader import get_config
import AnalyticsAndDBScripts.sql_connect as sql
import AnalyticsAndDBScripts.sql_schemas as schema
import AnalyticsAndDBScripts.well_spacing as ws

In [ ]:
# Load credentials for SQL connection
sql_creds_dict = get_config('credentials', 'sql1_sa')

# Add db_name to the dictionary
sql_creds_dict['db_name'] = 'Analytics'

# Load parameters
params = get_config('well_spacing')

In [ ]:
# Statement to load data from SQL Server into a pandas dataframe
data_source = 'ENVERUS'
intersection_fraction = params['intersection_fraction']
comparison_field = 'FitGroup' # 'Interval' or 'FitGroup'
cond1 = f"C.DataSource = '{data_source}' AND W.FitGroup = 'NIOBRARA'"

statement1 = f'''
SELECT		W.WellID, W.API_UWI_Unformatted, W.FitGroup, W.Basin, C.Interval, W.FirstProdDate
FROM		dbo.WELL_HEADER W
INNER JOIN	dbo.COMPLETION_HEADER C
ON			W.CurrentCompletionID = C.CompletionID
WHERE		W.FirstProdDate IS NOT NULL
AND			W.Geometry IS NOT NULL
AND			{cond1}
'''

statement2 = f'''
WITH WELL_CTE AS ({statement1})
SELECT      S.WellID, S.neighboring_WellID, S.MinDistance, S.MedianDistance, S.AvgDistance, 
			S.Neighbor_IntersectionFraction, S.RelativePosition, W.FirstProdDate, 
   			W2.FirstProdDate AS neighbor_FirstProdDate, 
            DATEDIFF(DAY, W.FirstProdDate, W2.FirstProdDate) AS DaysToNeighborFirstProd
FROM        dbo.WELL_SPACING AS S
INNER JOIN  WELL_CTE AS W
ON          S.WellID = W.WellID
INNER JOIN  WELL_CTE AS W2
ON          S.neighboring_WellID = W2.WellID
WHERE       S.neighbor_IntersectionFraction >= {intersection_fraction}
AND 	   	W.Basin = W2.Basin
AND 	   	W.{comparison_field} = W2.{comparison_field}
'''

In [ ]:
# Execute query and store results in a dataframe
engine = sql.sql_connect(
    username=sql_creds_dict['username'], 
    password=sql_creds_dict['password'], 
    db_name=sql_creds_dict['db_name'], 
    server_name=sql_creds_dict['servername'], 
    port=sql_creds_dict['port']
)
try:
    well_df = pd.read_sql(statement1, engine)
    spacing_df = pd.read_sql(statement2, engine)
finally:
    engine.dispose()

In [ ]:
# Run parent_child_processing function
closest_wells = ws.parent_child_processing(
    spacing_df, 
    well_df, 
    co_completed_threshold=params['co_completed_threshold'], 
    id_col='WellID', 
    position_col='RelativePosition', 
    date_col='FirstProdDate', 
    distance_col='AvgDistance', 
    neighbor_date_col='neighbor_FirstProdDate', 
    scenario_name=data_source
)

In [ ]:
# Convert NaN to None for proper database insertion
closest_wells = closest_wells.where(pd.notnull(closest_wells), None)

In [ ]:
# Load well_df into dbo.PARENT_CHILD_STAGE table in SQL Server
sql.load_data_to_sql(closest_wells, sql_creds_dict, schema.parent_child_stage)

# Move data from dbo.PARENT_CHILD_STAGE to dbo.PARENT_CHILD and drop dbo.PARENT_CHILD_STAGE
sql.execute_stored_procedure(sql_creds_dict, 'sp_InsertFromStagingToParentChild')